In [1]:
import boto3
import sagemaker
import os
from pathlib import Path


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
session = sagemaker.Session()
bucket_name = 'sagemaker-mlops-demo-dev-2024'  # Tu bucket
s3 = boto3.client('s3')

In [3]:
training_script = '''
import tensorflow as tf
import argparse
import os
import numpy as np

def parse_args():
    parser = argparse.ArgumentParser()
    
    
    parser.add_argument('--epochs', type=int, default=5)
    parser.add_argument('--batch-size', type=int, default=32)
    parser.add_argument('--learning-rate', type=float, default=0.001)
    
    
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    
    return parser.parse_known_args()[0]

def load_data(train_dir, test_dir):
    """Cargar datos de entrenamiento y prueba"""
    train_images = np.load(os.path.join(train_dir, 'train_images.npy'))
    train_labels = np.load(os.path.join(train_dir, 'train_labels.npy'))
    test_images = np.load(os.path.join(test_dir, 'test_images.npy'))
    test_labels = np.load(os.path.join(test_dir, 'test_labels.npy'))
    
    return (train_images, train_labels), (test_images, test_labels)

def create_model():
    """modelo CNN"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

if __name__ == '__main__':
    args = parse_args()
    
    #data
    (train_images, train_labels), (test_images, test_labels) = load_data(args.train, args.test)
    
    
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=args.learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    
    model.fit(
        train_images, train_labels,
        batch_size=args.batch_size,
        epochs=args.epochs,
        validation_data=(test_images, test_labels)
    )
    
    
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
    print(f'Test accuracy: {test_accuracy}')
    
    
    model.save(os.path.join(args.model_dir, '1'))
'''


In [4]:
os.makedirs('code', exist_ok=True)
script_path = 'code/train.py'

with open(script_path, 'w') as f:
    f.write(training_script)

print("Script guardado localmente en:", script_path)


Script guardado localmente en: code/train.py


In [5]:
s3_key = 'mnist/code/train.py'
s3.upload_file(
    script_path,
    bucket_name,
    s3_key
)

print(f"\nScript subido a s3://{bucket_name}/{s3_key}")


Script subido a s3://sagemaker-mlops-demo-dev-2024/mnist/code/train.py
